In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import shutil

# Source and destination paths
source_path = "/kaggle/input/rulesdb/rules.db"
destination_path = "/kaggle/working/rules.db"

# Copy file
shutil.copy2(source_path, destination_path)

print(f"📂 File copied from {source_path} → {destination_path}")


In [ ]:
# Initialize LLM
!pip install -q transformers accelerate bitsandbytes sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# ✅ Model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Load model in 4-bit with CPU offload for T4 / smaller GPUs
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",                     # auto-assign layers to GPU/CPU
    load_in_4bit=True,                     # 4-bit quantization
    llm_int8_enable_fp32_cpu_offload=False, # offload layers to CPU if needed
)
model.eval()

# ✅ Prepare prompt
prompt = "Explain Machine Learning in a nutshell."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# ✅ Generate text
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        do_sample=False
    )

# ✅ Decode output
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n=== Mistral-7B-Instruct Output (4-bit) ===\n")
print(result)


In [ ]:
#Populate top_rules table Business Description by invoking LLM
import sqlite3
import json

# Connect to the database
conn = sqlite3.connect('/kaggle/working/rules.db')
cursor = conn.cursor()

# Fetch all records from rule_json_store
cursor.execute("SELECT key, rule_type, rulejson FROM top_rules where businessdescription is null and LENGTH(rulejson) < 100000 LIMIT 3000")
rows = cursor.fetchall()

print(f"🔍 Found {len(rows)} records to process.\n")

for i, (key, rule_type, rulejson) in enumerate(rows, 1):
    try:
        # Load existing JSON
        # data = json.loads(rulejson)
        # print(f"🔍  {rulejson} \n")
        # 🧩 Example modification (you can customize this logic):
        # Add or update a "lastUpdated" field
        # data["lastUpdated"] = "2025-10-31"

        # Optional: apply transformations, cleanup, re-extraction, etc.
        # e.g., data = extract(data, selector_json)

        # Convert back to string
        # updated_json = json.dumps(data, ensure_ascii=False, indent=2)

        prompt = f"Given the '{rulejson}' Pega '{rule_type}' rule JSON, summarize the rule in plain English. Explain what the rule does and when the rule triggers. Avoid technical JSON details unless necessary, and make the description readable for a business user."
        # print(f"⚠️ prompt: {prompt}")

        # Generate text
        # Tokenize input
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

        # Generate text
        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                temperature=0.1,
                top_p=0.9,
                do_sample=True
            )

        # Extract only the new tokens (excluding the input)
        generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
        result = tokenizer.decode(generated_tokens, skip_special_tokens=True)


        # Update the record
        cursor.execute("""
            UPDATE top_rules
            SET businessdescription = ?
            WHERE key = ?
        """, (result, key))
        # print(f"⚠️ response: {result}")
        conn.commit()
        print(f"✅ Updated ({i}/{len(rows)}) pzInsKey: {key}")

    except json.JSONDecodeError:
        print(f"⚠️ Skipped invalid JSON for pzInsKey: {key}")
        continue

# Commit all updates
# conn.commit()
conn.close()

print("\n🎉 All rulejson entries have been updated successfully.")
